# Segmenting and Clustering Neighbourhoods in Toronto

### Section: #1 - Explore and cluster neighbourhoods in Toronto

Capstone project - Battle of the Neighbourhoods. This project will compare suburbs and will determine similarities based on clustering techniques using location data services.

This project uses web scraping techniques to retrieve data from the Canadian Postal Code's Wikipedia page.

The data is then acquired and cleansed in preparation for clustering. 

We import a file that contains the geospatial loctions which we then merge with the post code data.

Install and import required libraries..

In [1]:
import pandas as pd
import numpy as np

!pip install bs4

from bs4 import BeautifulSoup
import requests

Setup the reference to the Postal Codes of Canada wiki page..

Read in the web page and define a Beautiful Soup object for manipulation

In [2]:
canadian_postcodes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
canadian_postcodes = requests.get(canadian_postcodes_url).text
soup = BeautifulSoup(canadian_postcodes, "html5lib")


lets find the tables within the web page

In [3]:
table_contents = []
canadian_postcodes_table = soup.find("table")

Lets take a look at the raw html table..

In [4]:
canadian_postcodes_table

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%;">

<tbody><tr>
<td style="width:11%;">
<p>M1A<br/><span style="font-size:85%;">Not assigned</span>
</p>
</td>
<td style="width:11%;">
<p>M2A<br/><span style="font-size:85%;">Not assigned</span>
</p>
</td>
<td style="width:11%;">
<p>M3A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%;">
<p>M4A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%;">
<p>M5A<br/><span style="font-size:85%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a> / <a href="/wiki/Harbourfront,_Toronto" title="Harbourfront, Toronto">Harbourf

we're going to wrangle the data into a dataframe consisting of 3 columns..

PostalCode, Borough, Neighborhood

And we're going to remove those cells that contain Not Assigned, merge neighborhoods into a single postal code area and remove the duplicate postal codes

In [5]:
for row in canadian_postcodes_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass #ignore these ones
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

#merge neighborhoods
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#remove duplicate values - some postal code area have multiple neighbourhoods i.e.M5A
postcode_data = df.drop_duplicates()

#check de-duped
#postcode_data.loc[df['PostalCode'] == 'M5A']



view the data

In [6]:
postcode_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Answer for Section 1 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

print the number of rows in the data

In [7]:
postcode_data.shape

(103, 3)

###Section: #2 - combining the Postal Code data with Geospatial data


Reading the Geospatial_Coordinates csv file from notebook project assets.... tried using the Geocoder package, but received errors, so CSV it is...

this cell will be hidden, however the data is read into a dataframe named geospatial_data and it will be referenced later in the notebook

In [8]:
# The code was removed by Watson Studio for sharing.

Lets review the Geospatial data

In [9]:
geospatial_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


print the number of rows in the data

In [10]:
geospatial_data.shape

(103, 3)

We're going to merge (or join) the post code data and the geospatial data so that we can link up the latitudes and longitudes for the neighbourhoods in Canada.

In [11]:
geospatial_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
merged_geospatial_data = pd.merge(postcode_data,geospatial_data,on='PostalCode')


Answer for Section 2 output the dataframe data

In [12]:
merged_geospatial_data.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
